# Summarise and plot the perpixel phenometrics

In this notebook we plot the long-term average phenology across Australia, the long-term trends, and a number of plots that show the phenology time-series across Australia to show examples of change.

In [ ]:
import os
import sys
import xvec
import pickle
import textwrap
import warnings
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sb
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from odc.geo.xr import assign_crs
import matplotlib.colors as colors
import matplotlib.ticker as ticker
from odc.geo.geom import Geometry
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords
%matplotlib inline

## Open per pixel phenology data

In [ ]:
lin_or_circ = 'circular'

p_average = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel_{lin_or_circ}_final.nc'), crs='EPSG:4326')
p_trends = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel_{lin_or_circ}_final.nc'), crs='EPSG:4326')

gdf = gpd.read_file('/g/data/os22/chad_tmp/Aus_phenology/data/vectors/IBRAv7_subregions_modified.geojson')

## Plot number of seasons

and generate an additional mask to remove the small percentage of pixels that failed in the trend analysis

In [ ]:
season_per_year = p_average['n_seasons']/p_average['n_years']
non_seasonal = xr.where((season_per_year <= 0.90),1,0)
extra_seasonal = xr.where((season_per_year >= 1.1),1,0)
seasonality_mask = (non_seasonal | extra_seasonal)

sos_nan = np.isnan(p_trends['SOS_slope']).where(~np.isnan(p_average['SOS']))
sos_nan = xr.where((sos_nan==1) & (seasonality_mask==0), 1, 0)

eos_nan = np.isnan(p_trends['EOS_slope']).where(~np.isnan(p_average['EOS']))
eos_nan = xr.where((eos_nan==1) & (seasonality_mask==0), 1, 0)

pos_nan = np.isnan(p_trends['POS_slope']).where(~np.isnan(p_average['POS']))
pos_nan = xr.where((pos_nan==1) & (seasonality_mask==0), 1, 0)

extra_mask = (seasonality_mask | sos_nan | eos_nan | pos_nan)

In [ ]:
# xr.where(extra_mask==1, 1, -1).odc.explore(
#     tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#            attr = 'Esri',
#            name = 'Esri Satellite',
# )

In [ ]:
# gdf.explore(column='SUB_NAME_7',
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             control = True,
#             legend=False
#            )

In [ ]:
locs =['Maraling','Simpson Desert', 'Monaro']

gdf_sub =  gdf[gdf['SUB_NAME_7'].isin(locs)]

In [ ]:
percent_masked = (seasonality_mask.where(seasonality_mask==1).sum() / (~np.isnan(season_per_year)).sum() * 100).item()
print(f'{percent_masked:.1f} % of Australia is classified as "non-annual seasonality"')

In [ ]:
# vmin,vmax=33,45
vmin,vmax=0.7,1.3
cmap='RdYlBu'
fig,ax=plt.subplots(1,2, figsize=(11,6), layout='constrained')
season_per_year.plot(add_colorbar=False, add_labels=False, cmap=cmap, vmin=vmin, vmax=vmax, ax=ax[0])

gdf_sub.plot(ax=ax[0], color=None, facecolor="none", edgecolor='k',linewidth=0.5)
gdf_sub.apply(lambda x: ax[0].annotate(text=textwrap.fill(x['SUB_NAME_7'], 12),
                        xy=x.geometry.centroid.coords[0], 
                        xytext=(x.geometry.centroid.coords[0][0], x.geometry.centroid.coords[0][1]+2.5),
                        ha='center', fontsize=9), axis=1)

ctx.add_basemap(ax[0], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
ax[0].axes.get_xaxis().set_ticks([])
ax[0].axes.get_yaxis().set_ticks([])

#need to create colorbar manually to have label on top
norm = colors.Normalize(vmin=vmin, vmax=vmax)
cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
ax_cbar = fig.colorbar(cbar, ax=ax[0], shrink=0.6)
ax_cbar.ax.set_title('growth\ncycles\nyr⁻¹', fontsize=8)
ax[0].set_title('Average growth cycles per year');

seasonality_mask.where(seasonality_mask==1).plot(add_colorbar=False, add_labels=False,ax=ax[1])
ctx.add_basemap(ax[1], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
ax[1].axes.get_xaxis().set_ticks([])
ax[1].axes.get_yaxis().set_ticks([])
ax[1].set_title('Non-annual variation (cycles yr⁻¹ <=0.90 or >= 1.1)');

fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/num_seasons_AusENDVI-clim_MCD43A4.png',
            bbox_inches='tight', dpi=300);


### Average phenology

In [ ]:
fig,axes=plt.subplots(4,3, figsize=(11,11), sharey=True, sharex=True, layout='constrained')

pheno_stats=[
    'SOS', 'vSOS', 'AOS',
    'POS', 'vPOS', 'ROG',
    'EOS', 'vEOS', 'ROS',
    'LOS', 'vTOS', 'IOS'
]

cmaps = ['twilight', 'gist_earth_r', 'plasma',
         'twilight', 'gist_earth_r','magma',
         'twilight', 'gist_earth_r', 'magma_r',
         'viridis', 'gist_earth_r', 'inferno'
        ]
for ax,pheno,cmap in zip(axes.ravel(), pheno_stats, cmaps):
    if "v" not in pheno:
        vmin, vmax=0, 365
        label='DOY'
    if "v" in pheno:
        vmin,vmax=0.1, 0.85
        label='NDVI'
    if 'LOS' in pheno:
        vmin, vmax=160, 300
        label='Days'
    if 'AOS' in pheno:
        vmin, vmax=0.05, 0.4
        label='NDVI'
    if 'IOS' in pheno:
        vmin, vmax=20, 180
        label='NDVI\nyr⁻¹'
    if 'ROG' in pheno:
        vmin, vmax=0.00025, 0.0025
        label='NDVI\nday⁻¹'
    if 'ROS' in pheno:
        vmin, vmax=-0.0025, -0.00025
        label='NDVI\nday⁻¹'
    im=p_average[pheno].where(seasonality_mask!=1).plot(ax=ax, add_colorbar=False, cmap=cmap, vmin=vmin, vmax=vmax, add_labels=False)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    #need to create colorbar manually to have label on top
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    ax_cbar = fig.colorbar(cbar, ax=ax, shrink=0.7)
    ax_cbar.ax.set_title(label, fontsize=8)
    ax.set_title(f'{pheno}')

fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/average_phenometrics_perpixel_AusENDVI.png',
            bbox_inches='tight', dpi=300)

### Circular standard deviation

In [ ]:
fig,ax = plt.subplots(1,3, figsize=(15,5), sharey=True, layout='constrained')

p_average_circ['SOS_std'].where(seasonality_mask!=1).plot(vmin=0, vmax=100, cmap='magma', ax=ax[0], add_colorbar=False, add_labels=False)
ctx.add_basemap(ax[0], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
ax[0].set_title(f'mean std. dev. in SOS: {p_average_circ["SOS_std"].where(seasonality_mask!=1).mean():.1f} days')
ax[0].axes.get_xaxis().set_ticks([])
ax[0].axes.get_yaxis().set_ticks([])

p_average_circ['POS_std'].where(seasonality_mask!=1).plot(vmin=0, vmax=100, cmap='magma', ax=ax[1],  add_colorbar=False, add_labels=False)
ctx.add_basemap(ax[1], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
ax[1].set_title(f'mean std. dev. in POS: {p_average_circ["POS_std"].where(seasonality_mask!=1).mean():.1f} days')
ax[1].axes.get_xaxis().set_ticks([])
ax[1].axes.get_yaxis().set_ticks([])

p_average_circ['EOS_std'].where(seasonality_mask!=1).plot(vmin=0, vmax=100, cmap='magma', ax=ax[2],  add_colorbar=False, add_labels=False)
ctx.add_basemap(ax[2], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
ax[2].set_title(f'mean std. dev. in EOS: {p_average_circ["EOS_std"].where(seasonality_mask!=1).mean():.1f} days')
ax[2].axes.get_xaxis().set_ticks([])
ax[2].axes.get_yaxis().set_ticks([])

#need to create colorbar manually to have label on top
norm = colors.Normalize(vmin=0, vmax=100)
cbar = plt.cm.ScalarMappable(norm=norm, cmap='magma')
ax_cbar = fig.colorbar(cbar, ax=ax[1], shrink=0.6)
ax_cbar.ax.set_title('circular\nstd.dev.\n(days)', fontsize=8);

### Phenology trends

This is inordinately slow owing to the significance hatch plotting.

In [ ]:
fig,axes=plt.subplots(4,3, figsize=(11,11),  layout='constrained')#sharey=True, sharex=True,
pheno_stats=[
    'SOS', 'vSOS', 'AOS',
    'POS', 'vPOS', 'ROG',
    'EOS', 'vEOS', 'ROS',
    'LOS', 'vTOS', 'IOS'
]
cmaps = ['coolwarm','BrBG','PiYG',
         'coolwarm','BrBG','Spectral',
         'coolwarm','BrBG','Spectral_r',
         'PRGn', 'BrBG','PuOr'
        ]
for ax,pheno,cmap in zip(axes.ravel(), pheno_stats, cmaps):
   
    if "v" not in pheno:
        vmin, vmax=-2.0,2.0
        label='days\nyr⁻¹'
    if "v" in pheno:
        vmin,vmax=-0.0025, 0.0025
        label='NDVI\nyr⁻¹'
    if 'LOS' in pheno:
        vmin, vmax=-2.0, 2.0
        label='days\nyr⁻¹'
    if 'AOS' in pheno:
        vmin, vmax=-0.0025, 0.0025
        label='NDVI\nyr⁻¹'
    if 'ROG' in pheno:
        vmin, vmax=-2.5e-05, 2.5e-05
        label='NDVI\nday⁻¹ yr⁻¹'
    if 'ROS' in pheno:
        vmin, vmax=-2.5e-05, 2.5e-05
        label='NDVI\nday⁻¹ yr⁻¹'
    if 'IOS' in pheno:
        vmin, vmax=-1, 1
        label='NDVI\nyr⁻¹'

    if any(pheno in x for x in ['SOS', 'POS', 'EOS', 'LOS']):
        d_to_plot = (p_trends[pheno+'_slope'] * season_per_year).where(extra_mask!=1)
    
    else:
        d_to_plot = (p_trends[pheno+'_slope'] * season_per_year).where(seasonality_mask!=1)
    
    im=d_to_plot.plot(ax=ax, add_colorbar=False, cmap=cmap, vmin=vmin, vmax=vmax, add_labels=False)
                             
    # significance plotting
    lons, lats = np.meshgrid(d_to_plot.longitude, d_to_plot.latitude)
    sig_area = np.where(p_trends[pheno+'_p_value'] <= 0.05)
    ax.hexbin(x=lons[sig_area].reshape(-1),
             y=lats[sig_area].reshape(-1),
             C=d_to_plot.where(p_trends[pheno+'_p_value'] <= 0.05).data[sig_area].reshape(-1),
             hatch='....',
             alpha=0,
             gridsize=(400,400)
            )    

    ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])

    #need to create colorbar manually to have label on top
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    ax_cbar = fig.colorbar(cbar, ax=ax, shrink=0.7)
    ax_cbar.ax.set_title(label, fontsize=8)
    ax.set_title(f'{pheno} trend')
    # print(pheno)
    
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/Trends_phenometrics_perpixel_AusENDVI.png',
            bbox_inches='tight', dpi=300)

## Percent change

In [ ]:
for var in ['SOS', 'POS', 'EOS', 'LOS']:
    trends = p_trends[var+'_slope'].where(extra_mask!=1)
    pval = p_trends[var+'_p_value'].where(extra_mask!=1)
    
    total_nonnan = (~np.isnan(trends)).sum().values.item()
    total_significant = (~np.isnan(trends.where(pval<=0.05))).sum()
    significant_positive = (~np.isnan(trends.where(trends>0).where(pval <=0.05))).sum().values.item()
    significant_negative = (~np.isnan(trends.where(trends<0).where(pval <=0.05))).sum().values.item()
    
    total_percent_changed = total_significant / total_nonnan * 100
    total_percent_positive_change = significant_positive / total_nonnan * 100
    total_percent_negative_change = significant_negative / total_nonnan * 100
    
    print(f'{total_percent_changed:.2f} % of valid pixels in Australia has seen significant change in {var}')
    print(f'{total_percent_positive_change:.2f} % of valid pixels in Australia has seen significant positive trends in {var}')
    print(f'{total_percent_negative_change:.2f} % of valid pixels in Australia has seen significant negative trends in {var}')
    print('\n')


In [ ]:
for var in ['vSOS', 'vPOS', 'vEOS', 'AOS', 'ROG', 'ROS', 'IOS']:
    trends = p_trends[var+'_slope'].where(seasonality_mask!=1)
    pval = p_trends[var+'_p_value'].where(seasonality_mask!=1)
    
    total_nonnan = (~np.isnan(trends)).sum().values.item()
    total_significant = (~np.isnan(trends.where(pval<=0.05))).sum()
    significant_positive = (~np.isnan(trends.where(trends>0).where(pval <=0.05))).sum().values.item()
    significant_negative = (~np.isnan(trends.where(trends<0).where(pval <=0.05))).sum().values.item()
    
    total_percent_changed = total_significant / total_nonnan * 100
    total_percent_positive_change = significant_positive / total_nonnan * 100
    total_percent_negative_change = significant_negative / total_nonnan * 100
    
    print(f'{total_percent_changed:.2f} % of valid pixels in Australia have seen significant change in {var}')
    print(f'{total_percent_positive_change:.2f} % of valid pixels in Australia have seen significant positive trends in {var}')
    print(f'{total_percent_negative_change:.2f} % of valid pixels in Australia have seen significant negative trends in {var}')
    print('\n')

## Facet plots of phenograms

For a selection of IBRA subregions, show phenograms.

In [ ]:
# gdf.explore(column='SUB_NAME_7',
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             control = True,
#             legend=False
#            )

In [ ]:
product='AusENDVI-clim_MCD43A4'
timeseries_file = '/g/data/os22/chad_tmp/Aus_phenology/data/pickle/IBRA_subregions_NDVI_AusENDVI-clim_MCD43A4.pkl'
with open(timeseries_file, 'rb') as f:
    results = pickle.load(f)

#open a simple landcover map
lc = xr.open_dataarray('/g/data/os22/chad_tmp/Aus_phenology/data/ESA_WorldCover_processed.nc')
mask = ~np.isnan(p_trends.vPOS_slope)
lc = lc.where(mask)

In [ ]:
layout = [
    ['A',  'B',    'C', 'D'],
    ['E','center', 'center', 'F'],
    ['G','center', 'center', 'H'],
    ['I',  'J',    'K', 'L']
]

locs =[
    'Mitchell','Wilto', 'Weipa Plateau','Daintree-Bloomfield', #Birdum ''
    'Geraldton Hills rainfed crop', 'Townsville Plains', #''Northern Jarrah Forest
    'Warre', 'Dalmorto',  #'Southern Jarrah Forest'
    'Eastern Mallee','Carlisle', 'Southern Ranges', 'East Gippsland Lowlands' #'','East Gippsland Lowlands'
      ] # '

# Create the mosaic plot
fig, axes = plt.subplot_mosaic(layout, figsize=(15, 12))

subax=[
    axes['A'], axes['B'],axes['C'],axes['D'],
    axes['E'],axes['F'],axes['G'], axes['H'],
    axes['I'],axes['J'],axes['K'], axes['L']
      ]

#centre image of basic landcover
categories =['tree', 'shrub', 'grass', 'sparse', 'crop'] #'irrigated'
colors = ['#006400','#ffbb22', '#ffff4c', '#b4b4b4', '#f096ff'] #'#fa0000' 
cmap = LinearSegmentedColormap.from_list("landcover_cmap", colors, N=len(colors))

im = lc.plot(cmap=cmap, ax=axes['center'], add_colorbar=False, add_labels=False)
gdf[gdf['SUB_NAME_7'].isin(locs)].plot(ax=axes['center'], color=None, facecolor="none", edgecolor='k',linewidth=1)

axins1 = inset_axes(axes['center'],width="65%",height="5%",loc="lower left", borderpad=2.85)
cbar = fig.colorbar(im,cax=axins1, ticks=[1.45,2.25,3,3.85,4.65],orientation='horizontal')
cbar.ax.set_xticklabels(categories)

ctx.add_basemap(axes['center'], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
axes['center'].set_yticklabels([])
axes['center'].set_xticklabels([])
axes['center'].set_title(None)

#create a grid
xi = np.linspace(1, 12, 12) #months
yi = np.linspace(1982, 2022, 41) #years
Xi, Yi = np.meshgrid(xi, yi) #grid coords

for ax,k,v in zip(subax, locs, [results[x] for x in locs]):
    dss = v.resample(time='MS').mean()
    zi = griddata(points=(dss.time.dt.month, dss.time.dt.year),
                  values=dss.values,
                  xi=(xi[None, :], yi[:, None]),
                  method='nearest')
    
    sp = ax.contourf(zi, levels=75, cmap='gist_earth_r')
    clb = fig.colorbar(sp, ax=ax, shrink=0.75, format="%0.2f")
    clb.ax.set_title('NDVI', fontsize=8)
    ax.set(xticks=range(0,12))
    ax.set_xticklabels(["J","F","M","A","M","J","J","A","S","O","N","D"], fontsize=8);
    ax.set(yticks=range(0,41))
    ax.set_yticklabels([str(y) for y in range(1982,2023)], fontsize=8);
    ax.yaxis.set_major_locator(ticker.MultipleLocator(base=10))
    ax.set_title(k, fontsize=10);

fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/phenograms_headlinefigure.png',
          bbox_inches='tight', dpi=300);

## Example phenology timeseries for some regions

In [ ]:
phenometrics_file = '/g/data/os22/chad_tmp/Aus_phenology/data/pickle/IBRA_subregions_AusENDVI-clim_MCD43A4_phenometrics_new.pkl'
with open(phenometrics_file, 'rb') as f:
    eco_regions_phenometrics = pickle.load(f)

ndvi_path = '/g/data/os22/chad_tmp/AusENDVI/results/publication/AusENDVI-clim_MCD43A4_gapfilled_1982_2022_0.2.0.nc'
ds = xr.open_dataset(ndvi_path)['AusENDVI_clim_MCD43A4']

In [ ]:
layout = [
    ['A','A','A', 'B', 'B','B'],
    ['A','A','A', 'C', 'C','C'],
    ['D','D', 'D', 'E', 'E','E'],
    ['F',  'F','F', 'G', 'G', 'G']
]

locs =[
    'Birdum','Atarting', 'Murrumbidgee',
    'Carnarvon Ranges', 'St Vincent rainfed crop', 'Warre'
      ]

# Create the mosaic plot
# with plt.style.context('ggplot'):
fig, axes = plt.subplot_mosaic(layout, figsize=(18, 12), layout='constrained')

subax=[
    axes['B'],axes['C'],axes['D'],
    axes['E'],axes['F'], axes['G']
      ]

#image of basic landcover
categories =['tree', 'shrub', 'grass', 'sparse', 'crop'] #'irrigated'
colors = ['#006400','#ffbb22', '#ffff4c', '#b4b4b4', '#f096ff'] #'#fa0000' 
cmap = LinearSegmentedColormap.from_list("landcover_cmap", colors, N=len(colors))

im = lc.plot(cmap=cmap, ax=axes['A'], add_colorbar=False, add_labels=False)
gdf[gdf['SUB_NAME_7'].isin(locs)].plot(ax=axes['A'], color=None, facecolor="none", edgecolor='k',linewidth=1)

axins1 = inset_axes(axes['A'],width="65%",height="5%",loc="lower left", borderpad=2.85)
cbar = fig.colorbar(im,cax=axins1, ticks=[1.45,2.25,3,3.85,4.65],orientation='horizontal')
cbar.ax.set_xticklabels(categories)

ctx.add_basemap(axes['A'], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
axes['A'].set_yticklabels([])
axes['A'].set_xticklabels([])
axes['A'].set_title(None)

# label polygons
gdf[gdf['SUB_NAME_7'].isin(locs)].apply(lambda x: axes['A'].annotate(text=textwrap.fill(x['SUB_NAME_7'], 12),
                        xy=x.geometry.centroid.coords[0], 
                        xytext=(x.geometry.centroid.coords[0][0], x.geometry.centroid.coords[0][1]+1.5),
                        ha='center', fontsize=12), axis=1)

ctx.add_basemap(axes['A'], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
axes['A'].set_yticklabels([])
axes['A'].set_xticklabels([])
axes['A'].set_title(None)

for ax,k,v in zip(subax, locs, [results[x] for x in locs]):
    v.plot(ax=ax, color='tab:blue', linestyle='--', linewidth=1.0, label='Daily NDVI') #.sel(time=slice('1997','2016'))
    geom = Geometry(geom=gdf[gdf['SUB_NAME_7'].isin([k])].geometry.item(), crs=gdf.crs)
    dss = ds.odc.mask(poly=geom)
    dss = dss.mean(['latitude', 'longitude'])
    dss.plot.scatter(ax=ax, label='Monthly NDVI', alpha=0.9, facecolors='none', edgecolors='tab:gray')

    # ax.set_ylim(0.15, 0.7)
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].SOS.values, eco_regions_phenometrics[k].SOS_year.values)],
               y=eco_regions_phenometrics[k].vSOS,
              c='tab:green', label='SOS', zorder=10)
    
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].EOS.values, eco_regions_phenometrics[k].EOS_year.values)],
               y=eco_regions_phenometrics[k].vEOS,
              c='tab:purple', label='EOS', zorder=10)
    
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].POS.values, eco_regions_phenometrics[k].POS_year.values)],
                   y=eco_regions_phenometrics[k].vPOS,
                  c='black', label='POS', zorder=10)
        
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].TOS.values, eco_regions_phenometrics[k].TOS_year.values)],
               y=eco_regions_phenometrics[k].vTOS,
              c='tab:orange', label='TOS', zorder=10)
    
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_title(k, fontsize=12)
    ax.grid()
    ax.set_yticklabels(['{:2.2f}'.format(x) for x in ax.get_yticks()])
    ax.set_xlim(pd.Timestamp("1996-01-1"), pd.Timestamp("2013-12-31"))
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/phenology_example_regions.png',
      bbox_inches='tight', dpi=300);

In [ ]:
# get handles and labels for reuse
label_params = axes['G'].get_legend_handles_labels() 

figl, axl = plt.subplots()
axl.axis(False)
axl.legend(*label_params, loc="best", fontsize=12, markerscale=2, ncol=6)
figl.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/legend_example_phenology.png',
          bbox_inches='tight', dpi=300);

## Summarise over IBRA polygons

In [ ]:
# ecoregions_file = '/g/data/os22/chad_tmp/Aus_phenology/data/vectors/IBRAv7_subregions_modified.geojson'
# gdf = gpd.read_file(ecoregions_file)

# p_average_gdf = p_average.xvec.zonal_stats(
#     gdf.geometry, x_coords="longitude", y_coords="latitude", stats="mean"
# ).xvec.to_geodataframe()

# p_trends_gdf = p_trends.xvec.zonal_stats(
#     gdf.geometry, x_coords="longitude", y_coords="latitude", stats="mean"
# ).xvec.to_geodataframe()

In [ ]:
# fig,axes=plt.subplots(4,3, figsize=(11,11), sharey=True, sharex=True, layout='constrained')
# pheno_stats=['SOS','vSOS', 'LOS', 
#              'POS', 'vPOS', 'ROG',
#              'EOS', 'vEOS', 'ROS',
#              'IOS', 'vTOS', 'AOS',
#       ]
# cmaps = ['twilight', 'gist_earth_r', 'viridis',
#          'twilight', 'gist_earth_r','magma',
#          'twilight', 'gist_earth_r', 'magma_r',
#          'inferno', 'gist_earth_r','plasma'
#         ]
# for ax,pheno,cmap in zip(axes.ravel(), pheno_stats, cmaps):
#     if "v" not in pheno:
#         vmin, vmax=0, 365
#         label='DOY'
#     if "v" in pheno:
#         vmin,vmax=0.1, 0.85
#         label='NDVI'
#     if 'LOS' in pheno:
#         vmin, vmax=160, 300
#         label='days'
#     if 'AOS' in pheno:
#         vmin, vmax=0.05, 0.4
#         label='NDVI'
#     if 'IOS' in pheno:
#         vmin, vmax=20, 200
#         label='NDVI/\n season'
#     if 'ROG' in pheno:
#         vmin, vmax=0.00025, 0.0025
#         label='NDVI/\nday'
#     if 'ROS' in pheno:
#         vmin, vmax=-0.0025, -0.00025
#         label='NDVI/\nday'
#     im=p_average_gdf.plot(column=pheno, ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax)
#     ctx.add_basemap(im, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
#     ax.axes.get_xaxis().set_ticks([])
#     ax.axes.get_yaxis().set_ticks([])
#     #need to create colorbar manually to have label on top
#     norm = colors.Normalize(vmin=vmin, vmax=vmax)
#     cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
#     ax_cbar = fig.colorbar(cbar, ax=ax, shrink=0.7)
#     ax_cbar.ax.set_title(label, fontsize=8)
#     ax.set_title(f'{pheno}, 1982-2022')

# fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/average_phenometrics_map_IBRA_AusENDVI-clim_MCD43A4.png',
#             bbox_inches='tight', dpi=300)

### Plot summarised trends

In [ ]:
# fig,axes=plt.subplots(4,3, figsize=(11,11),  layout='constrained')#sharey=True, sharex=True,
# pheno_stats=['SOS','vSOS', 'LOS', 
#              'POS', 'vPOS', 'ROG',
#              'EOS', 'vEOS', 'ROS',
#              'AOS', 'vTOS', 'IOS'
#       ]
# cmaps = ['coolwarm','BrBG','PRGn',
#          'coolwarm','BrBG','Spectral',
#          'coolwarm','BrBG','Spectral_r',
#          'PiYG','BrBG','PuOr'
#         ]

# for ax,pheno,cmap in zip(axes.ravel(), pheno_stats, cmaps):
   
#     if "v" not in pheno:
#         vmin, vmax=-1.5,1.5
#         label='days/\nyear'
#     if "v" in pheno:
#         vmin,vmax=-0.0015, 0.0015
#         label='NDVI/\nyear'
#     if 'LOS' in pheno:
#         vmin, vmax=-1.5, 1.5
#         label='days/\nyear'
#     if 'AOS' in pheno:
#         vmin, vmax=-0.002, 0.002
#         label='NDVI\nyear'
#     if 'ROG' in pheno:
#         vmin, vmax=-2.0e-05, 2.0e-05
#         label='NDVI/day/\nyear'
#     if 'ROS' in pheno:
#         vmin, vmax=-2.0e-05, 2.0e-05
#         label='NDVI/day/\nyear'
#     if 'IOS' in pheno:
#         vmin, vmax=-0.5, 0.5
#         label='NDVI/\nyear'

#     # im=pheno_trends.plot(column=pheno+'_slope', ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax)
    
#     significant = p_trends_gdf[p_trends_gdf[pheno+'_p_value'] <= 0.1]
#     not_significant = p_trends_gdf[p_trends_gdf[pheno+'_p_value'] > 0.1]
    
#     im=significant.plot(column=pheno+'_slope', ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax, hatch='....')     
#     not_significant.plot(column=pheno+'_slope', ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax)     

#     ctx.add_basemap(im, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
#     ax.axes.get_xaxis().set_ticks([])
#     ax.axes.get_yaxis().set_ticks([])
    
#     #need to create colorbar manually to have label on top
#     norm = colors.Normalize(vmin=vmin, vmax=vmax)
#     cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
#     ax_cbar = fig.colorbar(cbar, ax=ax, shrink=0.7)
#     ax_cbar.ax.set_title(label, fontsize=8)
#     ax.set_title(f'{pheno}, 1982-2022')
#     print(pheno)
    
# # axes[3,2].axis('off');
# fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/trends_phenometrics_map_IBRA_AusENDVI-clim_MCD43A4.png',
#             bbox_inches='tight', dpi=300)